# Plot Segmentation in Agriculture Using Computer Vision Techniques: A Scientific Approach

In the realm of precision agriculture, plot segmentation plays a pivotal role in crop management and yield optimization. This Jupyter Notebook presents a comprehensive workflow for segmenting agricultural plots by harnessing the power of computer vision and image processing techniques. We demonstrate the integration of image manipulation, filtering, and augmentation to extract meaningful insights from agricultural imagery.


# Section 1: Setup and Data Preparation


## Initial Imports

Here we import necessary libraries such as OpenCV, NumPy, and Matplotlib. These libraries provide us with the tools required for image manipulation and visualization.


In [ ]:
!pip install tensorflow -q

In [ ]:
import os
import sys
import math
import random
import time
import subprocess
import psutil

import cv2                          as cv
import keras                        as K
import numpy                        as np
import tensorflow                   as tf
import matplotlib.pyplot            as plt

from PIL                            import Image, ImageTk
from google.colab                   import drive
from scipy.ndimage                  import convolve

from tensorflow                     import data as tf_data
from keras                          import optimizers, callbacks, Model
from keras.models                   import Sequential
from keras.layers                   import Conv2D, Input, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Activation, SeparableConv2D, Conv2DTranspose, UpSampling2D, add

## Drive Mounting

Here we mount the Google Drive to access the dataset.

In [ ]:
drive.mount('/content/drive')

## Image Data Manager Class

The `ImageDataManager` class is designed to simplify the handling of image datasets. It loads images from specified directories, processes them into a usable format, and stores them for further analysis.

In [ ]:
class ImageDataManager:
    """
    A class used to manage image data for processing and analysis.

    Attributes
    ----------
    objects : dict
        A dictionary to store processed image data with mask IDs as keys.
    base_masks_path : str
        Path to the directory containing mask images.
    base_inputs_path : str
        Path to the directory containing input images.

    Methods
    -------
    process_images()
        Processes all images found in the base paths and populates the objects dictionary.
    process_input_folder(input_folder_path)
        Processes all images within a given input folder path and returns a list of images.
    """

    def __init__(self, base_masks_path, base_inputs_path):
        """
        Constructs all the necessary attributes for the ImageDataManager object.

        Parameters
        ----------
        base_masks_path : str
            Path to the directory containing mask images.
        base_inputs_path : str
            Path to the directory containing input images.
        """
        self.objects = {}
        self.base_masks_path = base_masks_path
        self.base_inputs_path = base_inputs_path
        self.process_images()

    @staticmethod
    def load_image(image_path, flags=cv.IMREAD_COLOR):
        """
        Loads an image from a specified path.

        Parameters
        ----------
        image_path : str
            Path to the image file to be loaded.
        flags : int
            Flags for image color format to be read.

        Returns
        -------
        ndarray
            The image loaded into memory.
        """
        return cv.imread(image_path, flags)

    @staticmethod
    def split_channels(image_path):
        """
        Splits the channels of an image at the given path.

        Parameters
        ----------
        image_path : str
            Path to the image file.

        Returns
        -------
        list
            A list containing the channels of the image.
        """
        image = cv.imread(image_path, cv.IMREAD_UNCHANGED)
        channels = cv.split(image)
        if len(channels) == 4:
            return channels[:3]  # Ignore alpha channel
        return channels

    def process_images(self):
        """
        Processes the images found in the base paths and stores them in the objects dictionary.
        """
        for mask_filename in os.listdir(self.base_masks_path):
            if mask_filename.endswith('.png'):
                mask_id = os.path.splitext(mask_filename)[0]
                mask_path = os.path.join(self.base_masks_path, mask_filename)
                mask = self.load_image(mask_path, cv.IMREAD_GRAYSCALE)
                input_folder_path = os.path.join(self.base_inputs_path, mask_id)
                images = self.process_input_folder(input_folder_path)
                self.objects[int(mask_id)] = {
                    'mask': mask,
                    'input': input_folder_path,
                    'images': images
                }

    def process_input_folder(self, input_folder_path):
        """
        Processes all images within the given input folder path.

        Parameters
        ----------
        input_folder_path : str
            Path to the input folder containing image files.

        Returns
        -------
        list
            A list of processed images.
        """
        images = []
        if os.path.isdir(input_folder_path):
            for image_filename in os.listdir(input_folder_path):
                image_path = os.path.join(input_folder_path, image_filename)
                if image_filename.endswith('.png'):
                    channels = self.split_channels(image_path)
                    images.append(np.array(channels))
                elif image_filename.endswith(('.tif', '.tiff')):
                    tif_image = self.load_image(image_path, cv.IMREAD_UNCHANGED)
                    images.append(np.array([tif_image]))
        return images

# Section 2: Visualization Tools


## Visualizer Class
The `Visualizer` class contains methods for displaying images and their channels. It helps us understand the data and the effects of processing steps visually.

In [ ]:
class Visualizer:
    """
    A class used to visualize image data.

    Methods
    -------
    visualize_objects(objects)
        Visualizes the processed image data from the objects dictionary.
    calculate_subplots(image_sets)
        Calculates the number of subplots needed for the visualization based on the image sets.
    display_images(num_rows, num_columns, image_sets)
        Displays images in a grid layout as subplots.
    """

    @staticmethod
    def visualize_objects(objects):
        """
        Visualizes the processed image data from the objects dictionary.

        Parameters
        ----------
        objects : dict
            Dictionary containing processed image data to visualize.
        """
        for mask_id, data in objects.items():
            mask = data['mask']
            image_sets = data['images']
            num_subplots = Visualizer.calculate_subplots(image_sets)
            num_columns = min(num_subplots, 4)
            num_rows = math.ceil(num_subplots / num_columns)
            plt.figure(figsize=(4 * num_columns, 3 * num_rows))
            plt.subplot(num_rows, num_columns, 1)
            plt.title(f"Mask {mask_id}")
            plt.imshow(mask, cmap='gray')
            plt.axis('off')
            Visualizer.display_images(num_rows, num_columns, image_sets)
            plt.tight_layout()
            plt.show()

    @staticmethod
    def calculate_subplots(image_sets):
        """
        Calculates the number of subplots needed based on the image sets.

        Parameters
        ----------
        image_sets : list
            A list of image sets, where each image set corresponds to a group of channels.

        Returns
        -------
        int
            The total number of subplots needed.
        """
        return 1 + sum(len(images) for images in image_sets)

    @staticmethod
    def display_images(num_rows, num_columns, image_sets):
        """
        Displays images in a grid layout as subplots.

        Parameters
        ----------
        num_rows : int
            Number of rows in the grid layout.
        num_columns : int
            Number of columns in the grid layout.
        image_sets : list
            A list of image sets to be displayed.
        """
        subplot_idx = 2
        for image_set in image_sets:
            for channel in image_set:
                plt.subplot(num_rows, num_columns, subplot_idx)
                plt.imshow(channel, cmap='gray')
                plt.axis('off')
                title = f"TIFF Image {subplot_idx-1}" if len(image_set) == 1 else f"PNG Ch {subplot_idx-1}"
                plt.title(title)
                subplot_idx += 1

# Section 3: Image Processing Pipeline

## Preprocessing Auxiliary Classes

Here we define a suite of image preprocessing classes that extend `BaseImageProcess`. Each class implements a specific image preprocessing technique, such as rotation, blurring, and thresholding, which are essential for the feature extraction phase of plot segmentation.



In [ ]:
class BaseImageProcess:
    """
    BaseImageProcess: A base class for image processing algorithms.

    This class provides a basic framework for implementing image processing algorithms and is intended to be subclassed.
    Subclasses should implement the `apply` method to perform specific image processing operations on an input image.
    """

    def apply(self, img, mask=None):
        """
        Placeholder for applying an image processing algorithm.

        Args:
            img: The input image to process.

        Returns:
            The processed image.
        """
        pass


class Rotate(BaseImageProcess):
    def __init__(self):
        self.angle = random.choice(list(range(-180, 181, 10)))

    def apply(self, img, mask=None):
        height, width = img.shape[:2]
        rotation_matrix = cv.getRotationMatrix2D((width / 2, height / 2), self.angle, 1)
        return cv.warpAffine(img, rotation_matrix, (width, height)), (
            cv.warpAffine(mask, rotation_matrix, (width, height))
            if mask is not None
            else None
        )


class BilateralFilter(BaseImageProcess):
    """
    BilateralFilter: Applies bilateral filtering to an image to reduce noise while keeping edges sharp.
    """

    def __init__(self, d=9, sigmaColor=75, sigmaSpace=75):
        self.d = d
        self.sigmaColor = sigmaColor
        self.sigmaSpace = sigmaSpace

    def apply(self, img, mask=None):
        return cv.bilateralFilter(img, self.d, self.sigmaColor, self.sigmaSpace), mask


class Translate(BaseImageProcess):
    """
    Applies translation to an image using random horizontal and vertical shifts.

    Attributes:
        dx (int): Horizontal shift, chosen randomly from a specified range.
        dy (int): Vertical shift, chosen randomly from a specified range.
    """

    def __init__(self):
        self.dx = random.choice([-10, -5, 0, 5, 10])
        self.dy = random.choice([-10, -5, 0, 5, 10])

    def apply(self, img, mask=None):
        translation_matrix = np.float32([[1, 0, self.dx], [0, 1, self.dy]])
        height, width = img.shape[:2]
        return cv.warpAffine(img, translation_matrix, (width, height)), (
            cv.warpAffine(mask, translation_matrix, (width, height))
            if mask is not None
            else None
        )


class Flip(BaseImageProcess):
    """
    Flips an image either horizontally, vertically, or both, based on a randomly selected flip type.

    Attributes:
        flip_type (int): Type of flip to apply; -1 for both axes, 0 for vertical, 1 for horizontal.
    """

    def __init__(self):
        self.flip_type = random.choice([-1, 0, 1])

    def apply(self, img, mask=None):
        return cv.flip(img, self.flip_type), (
            cv.flip(mask, self.flip_type) if mask is not None else None
        )


class BrightnessContrast(BaseImageProcess):
    """
    Adjusts the brightness and contrast of an image using random values.

    Attributes:
        alpha (float): Factor by which the contrast will be adjusted.
        beta (int): Value that will be added to the pixels for brightness adjustment.
    """

    def __init__(self):
        self.alpha = random.uniform(0.5, 1.5)
        self.beta = random.randint(-50, 50)

    def apply(self, img, mask=None):
        return cv.convertScaleAbs(img, alpha=self.alpha, beta=self.beta), mask


class MedianBlur(BaseImageProcess):
    """
    Applies median blurring to an image using a randomly chosen kernel size.

    Attributes:
        kernel_size (int): The size of the kernel used, selected randomly from a set of possible odd sizes.
    """

    def __init__(self):
        self.kernel_size = random.choice([3, 5, 7, 9, 11])

    def apply(self, img, mask=None):
        return cv.medianBlur(img, self.kernel_size), mask


class RandomGaussianBlur(BaseImageProcess):
    """
    Applies Gaussian blur filtering to an image with a randomly chosen kernel size.

    Attributes:
        kernel_size (int): Size of the Gaussian blur kernel, selected randomly.
    """

    def __init__(self):
        self.kernel_size = random.choice([3, 5, 7, 9, 11])

    def apply(self, img, mask=None):
        return cv.GaussianBlur(img, (self.kernel_size, self.kernel_size), 0), mask


class GaussianBlur(BaseImageProcess):
    """
    GaussianBlur: Applies Gaussian blur filtering to an image.

    This class provides an implementation of Gaussian blur filtering, commonly used to reduce image noise and detail.

    Attributes:
        kernel_size (int): Size of the kernel used for the Gaussian filter.
    """

    def __init__(self, kernel_size=5):
        self.kernel_size = kernel_size

    def apply(self, img, mask=None):
        return cv.GaussianBlur(img, (self.kernel_size, self.kernel_size), 0), mask


class BinaryThresh(BaseImageProcess):
    """
    BinaryThresh: Applies binary thresholding to an image.

    Binary thresholding converts an image to binary (black and white) based on a threshold value. Pixels above the
    threshold are set to the maximum value, and those below are set to zero.

    Attributes:
        thresh (int): Threshold value.
        max_val (int): Maximum value to use with the threshold.
    """

    def __init__(self, thresh=127, max_val=255):
        self.thresh = thresh
        self.max_val = max_val

    def apply(self, img, mask=None):
        _img = cv.cvtColor(img, cv.COLOR_BGR2GRAY) if len(img.shape) > 2 else img
        _, _img = cv.threshold(_img, self.thresh, self.max_val, cv.THRESH_BINARY)
        return _img, mask


class AdaptiveMeanThresh(BaseImageProcess):
    """
    AdaptiveMeanThresh: Applies adaptive mean thresholding to an image.

    Unlike simple thresholding, adaptive thresholding changes the threshold dynamically over the image based on local
    image characteristics.

    Attributes:
        block_size (int): Size of a pixel neighborhood used to calculate the threshold.
        c (int): Constant subtracted from the calculated mean or weighted mean.
    """

    def __init__(self, block_size=11, c=2):
        self.block_size = block_size
        self.c = c

    def apply(self, img, mask=None):
        _img = cv.cvtColor(img, cv.COLOR_BGR2GRAY) if len(img.shape) > 2 else img
        return (
            cv.adaptiveThreshold(
                _img,
                255,
                cv.ADAPTIVE_THRESH_MEAN_C,
                cv.THRESH_BINARY,
                self.block_size,
                self.c,
            ),
            mask,
        )


class AdaptiveGaussThresh(BaseImageProcess):
    """
    AdaptiveGaussThresh: Applies adaptive Gaussian thresholding to an image.

    This method uses a weighted sum of neighbourhood values where weights are a Gaussian window, which provides
    a more natural thresholding, especially under varying illumination.

    Attributes:
        block_size (int): Size of a pixel neighborhood used to calculate the threshold.
        c (int): Constant subtracted from the calculated weighted sum.
    """

    def __init__(self, block_size=11, c=2):
        self.block_size = block_size
        self.c = c

    def apply(self, img, mask=None):
        _img = cv.cvtColor(img, cv.COLOR_BGR2GRAY) if len(img.shape) > 2 else img
        return (
            cv.adaptiveThreshold(
                _img,
                255,
                cv.ADAPTIVE_THRESH_GAUSSIAN_C,
                cv.THRESH_BINARY,
                self.block_size,
                self.c,
            ),
            mask,
        )


class OtsuThresh(BaseImageProcess):
    """
    OtsuThresh: Applies Otsu's thresholding to automatically perform histogram shape-based image thresholding.

    This method is useful when the image contains two prominent pixel intensities and calculates an optimal threshold
    separating these two classes so that their combined spread (intra-class variance) is minimal.
    """

    def apply(self, img, mask=None):
        _img = cv.cvtColor(img, cv.COLOR_BGR2GRAY) if len(img.shape) > 2 else img
        _, _img = cv.threshold(_img, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)
        return _img, mask


class MorphDilate(BaseImageProcess):
    """
    MorphDilate: Applies morphological dilation to an image.

    Dilation increases the white region in the image or size of the foreground object. Commonly used to accentuate
    features.

    Attributes:
        kernel_size (int): Size of the structuring element.
        iterations (int): Number of times dilation is applied.
    """

    def __init__(self, kernel_size=3, iterations=2):
        self.kernel_size = kernel_size
        self.iterations = iterations
        self.kernel = np.ones((self.kernel_size, self.kernel_size), np.uint8)

    def apply(self, img, mask=None):
        return cv.dilate(img, self.kernel, iterations=self.iterations), mask


class MorphErode(BaseImageProcess):
    """
    MorphErode: Applies morphological erosion to an image.

    Erosion erodes away the boundaries of the foreground object and is used to diminish the features of an image.

    Attributes:
        kernel_size (int): Size of the structuring element.
        iterations (int): Number of times erosion is applied.
    """

    def __init__(self, kernel_size=3, iterations=2):
        self.kernel_size = kernel_size
        self.iterations = iterations
        self.kernel = np.ones((self.kernel_size, self.kernel_size), np.uint8)

    def apply(self, img, mask=None):
        return cv.erode(img, self.kernel, iterations=self.iterations), mask


class LoG(BaseImageProcess):
    """
    LoG: Applies Laplacian of Gaussian filtering to an image.

    This method is used to highlight regions of rapid intensity change and is therefore often used for edge detection.
    First, it applies a Gaussian blur, then computes the Laplacian of the result.

    Attributes:
        sigma (float): Standard deviation of the Gaussian filter.
        size (int): Size of the filter kernel.
    """

    def __init__(self, sigma=2.0, size=None):
        self.sigma = sigma
        self.size = (
            size
            if size is not None
            else int(6 * self.sigma + 1) if self.sigma >= 1 else 7
        )
        if self.size % 2 == 0:
            self.size += 1

    def apply(self, img, mask=None):
        x, y = np.meshgrid(
            np.arange(-self.size // 2 + 1, self.size // 2 + 1),
            np.arange(-self.size // 2 + 1, self.size // 2 + 1),
        )
        kernel = (
            -(1 / (np.pi * self.sigma**4))
            * (1 - ((x**2 + y**2) / (2 * self.sigma**2)))
            * np.exp(-(x**2 + y**2) / (2 * self.sigma**2))
        )
        kernel = kernel / np.sum(np.abs(kernel))
        return cv.filter2D(img, -1, kernel), mask


class LoGConv(BaseImageProcess):
    """
    LoGConv: Implements convolution with a Laplacian of Gaussian kernel to an image.

    Similar to the LoG class, but tailored for applying custom convolution operations directly with a manually
    crafted LoG kernel.

    Attributes:
        sigma (float): Standard deviation of the Gaussian filter.
        size (int): Size of the filter kernel.
    """

    def __init__(self, sigma=2.0, size=None):
        self.sigma = sigma
        self.size = size if size is not None else int(6 * sigma + 1)
        if self.size % 2 == 0:
            self.size += 1

    def apply(self, img, mask=None):
        if len(img.shape) == 3:
            img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
        x, y = np.meshgrid(
            np.arange(-self.size // 2 + 1, self.size // 2 + 1),
            np.arange(-self.size // 2 + 1, self.size // 2 + 1),
        )
        kernel = (
            -(1 / (np.pi * self.sigma**4))
            * (1 - ((x**2 + y**2) / (2 * self.sigma**2)))
            * np.exp(-(x**2 + y**2) / (2 * self.sigma**2))
        )
        kernel = kernel / np.sum(np.abs(kernel))
        if len(img.shape) == 3:
            img = cv.cvtColor(img, cv.COLOR_GRAY2BGR)
            img = convolve(img, kernel)
            img = np.clip(img, 0, 255).astype(np.uint8)
        else:
            img = convolve(img, kernel)
        return img, mask

## Processing Pipeline Class

The `ProcessingPipeline` class orchestrates the application of filters and augmentations to the image data. It automates the process of image enhancement and prepares the data for segmentation.

In [ ]:
class ProcessingPipeline:
    """
    Manages the application of image processing filters and augmentations.

    Attributes:
        filters (list): A list of filter objects to apply to the images.
        augmentations (list): A list of augmentation objects to apply to the images.
        history (list): Records outcomes of applied filters and augmentations for visualization.
    """

    def __init__(self, plot_storyline=False):
        """
        Initializes the processing pipeline with empty lists for filters, augmentations, and history.
        """
        self.filters = []
        self.augmentations = []
        self.history = []

    def add_filters(self, filters):
        """
        Adds multiple filter objects to the pipeline.

        Parameters:
            filters (list): List of filter objects to be added.
        """
        self.filters.extend(filters)

    def clear_filters(self):
        """
        Clears all filter objects from the pipeline.
        """
        self.filters = []

    def add_augmentations(self, augmentations):
        """
        Adds multiple augmentation objects to the pipeline.

        Parameters:
            augmentations (list): List of augmentation objects to be added.
        """
        self.augmentations.extend(augmentations)

    def clear_augmentations(self):
        """
        Clears all augmentation objects from the pipeline.
        """
        self.augmentations = []

    def apply_filters(self, img):
        """
        Applies each filter in sequence to the image.

        Parameters:
            img (numpy.ndarray): The original image to be processed.

        Returns:
            numpy.ndarray: The image processed by all filters.
        """
        _img = img
        for _filter in self.filters:
            _img, _ = _filter.apply(_img, None)
            self.history.append((_img.copy(), type(_filter).__name__, "Filter"))
        return _img

    def apply_crop(self, img, mask, new_width=120, new_height=120):
        """
        Randomly crops the given image and mask arrays into 'n' new images and masks with dimensions 'new_width' x 'new_height'.

        Parameters:
            img (numpy.ndarray): The numpy array representing the original image.
            mask (numpy.ndarray): The numpy array representing the original mask.
            new_width (int): The width of the new images and masks.
            new_height (int): The height of the new images and masks.

        Returns:
            tuple: A tuple containing three elements:
                   - A list of numpy.ndarray representing the cropped images.
                   - A list of numpy.ndarray representing the cropped masks.
                   - A list of tuples containing the coordinates of each crop.
        """
        original_height, original_width = img.shape[:2]
        if new_width > original_width or new_height > original_height:
            raise ValueError(
                "New dimensions must be smaller than the original dimensions."
            )

        cropped_images, cropped_masks, crop_coordinates = [], [], []

        number_of_height_crops = original_height // new_height
        number_of_width_crops = original_width // new_width

        for row in range(0, number_of_height_crops):
            for col in range(0, number_of_width_crops):
                top = row * new_height
                left = col * new_width
                cropped_img = img[top : top + new_height, left : left + new_width]
                cropped_mask = mask[top : top + new_height, left : left + new_width]
                cropped_images.append(cropped_img)
                cropped_masks.append(cropped_mask)
                crop_coordinates.append(
                    ((left, top), (left + new_width, top + new_height))
                )

                self.history.append(
                    (cropped_img, f"Cropped Image at ({left}, {top})", "Crop")
                )

        return cropped_images, cropped_masks, crop_coordinates

    def apply_augmentations(self, images, masks, n=3):
        """
        Applies data augmentation to a list of images and their corresponding masks.

        Parameters:
            images (list of numpy.ndarray): The list of numpy arrays representing the original images.
            masks (list of numpy.ndarray): The list of numpy arrays representing the masks for the images.
            n (int): Number of augmentations to apply to each image.
            filters (list): List of instantiated filter classes to apply.

        Returns:
            tuple: A tuple containing two elements:
                   - List of numpy.ndarray representing the original and augmented images.
                   - List of numpy.ndarray representing the original and augmented masks.
        """
        all_images = []
        all_masks = []

        for image, mask in zip(images, masks):
            augmented_images = [image]
            augmented_masks = [mask]
            previous_transformations = set()

            while len(augmented_images) - 1 < n:
                selected_filter = random.choice(self.augmentations)
                transformation_key = (
                    type(selected_filter).__name__,
                    tuple(selected_filter.__dict__.values()),
                )

                if transformation_key not in previous_transformations:
                    augmented_image, augmented_mask = selected_filter.apply(image, mask)
                    augmented_images.append(augmented_image)
                    augmented_masks.append(augmented_mask)
                    previous_transformations.add(transformation_key)
                    self.history.append(
                        (
                            augmented_image,
                            f"Augmented with {type(selected_filter).__name__}",
                            "Augmentation",
                        )
                    )
                    self.history.append(
                        (
                            augmented_mask,
                            f"[MASK] Augmented with {type(selected_filter).__name__}",
                            "Augmentation",
                        )
                    )

            all_images.extend(augmented_images)
            all_masks.extend(augmented_masks)

        return all_images, all_masks

    def apply_normalization(self, imgs, masks):
        """
        Normalizes the pixel values of images and masks to the range [0, 1].

        Parameters:
            imgs (list of numpy.ndarray): The list of images to be normalized.
            masks (list of numpy.ndarray): The list of masks to be normalized.

        Returns:
            tuple: A tuple containing two elements:
                   - List of numpy.ndarray representing the normalized images.
                   - List of numpy.ndarray representing the normalized masks.
        """
        _imgs = []
        _masks = []
        for index, _img in enumerate(imgs):
            height, width, _ = _img.shape
            m_height, m_width = masks[index].shape
            norm_img = np.zeros((height, width))
            norm_mask = np.zeros((m_height, m_width))
            norm_img = cv.normalize(_img, norm_img, 0, 255, cv.NORM_MINMAX)
            norm_mask = cv.normalize(masks[index], norm_mask, 0, 255, cv.NORM_MINMAX)
            norm_img = norm_img / 255
            norm_mask = norm_mask / 255
            _imgs.append(norm_img)
            _masks.append(norm_mask)
        return _imgs, _masks

    def run(self, img, mask, n_augmented, crop_size=120, n_crop=20):
        """
        Executes the entire image processing pipeline.

        Parameters:
            img (numpy.ndarray): The original image to process.
            mask (numpy.ndarray): The associated mask for the image.
            n_augmented (int): The number of augmented images to generate.
            crop_size (int): The size for cropping the images.
            n_crop (int): The number of crops to produce.

        Returns:
            tuple: A tuple containing three elements:
                   - List of numpy.ndarray representing the normalized images.
                   - List of numpy.ndarray representing the normalized masks.
                   - List of tuples containing the coordinates of cropped areas.
        """
        highlighted_img = self.apply_filters(img)

        cropped_imgs, cropped_masks, cropped_coordinates = self.apply_crop(
            highlighted_img, mask, new_width=crop_size, new_height=crop_size
        )

        augmented_imgs, augmented_masks = self.apply_augmentations(
            cropped_imgs, cropped_masks, n_augmented
        )

        normalized_imgs, normalized_masks = self.apply_normalization(
            augmented_imgs, augmented_masks
        )

        return normalized_imgs, normalized_masks, cropped_coordinates

    def get_history(self):
        return self.history


# Section 4: Auxiliary Functions

## Helper Functions

Auxiliary functions such as `get_dataset` are defined here. These functions support the main pipeline by providing image loading capabilities and visualizing the processing history.



In [ ]:
def get_dataset(batch_size,
                img_size,
                input_img_arr,
                target_img_arr,
                max_dataset_len=None):
    """
    Returns a TensorFlow Dataset.

    Args:
        batch_size (int): The batch size for the dataset.
        img_size (tuple): The size of the images in the dataset.
        input_img_arr (numpy.array): Array of input images.
        target_img_arr (numpy.array): Array of target images.
        max_dataset_len (int, optional): Maximum length of the dataset. Defaults to None.

    Returns:
        tf.data.Dataset: A TensorFlow dataset batched with the specified batch size.
    """
    if max_dataset_len:
        input_img_arr = input_img_arr[:max_dataset_len]
        target_img_arr = target_img_arr[:max_dataset_len]
    dataset = tf_data.Dataset.from_tensor_slices((input_img_arr, target_img_arr))
    return dataset.batch(batch_size)

def visualize_dataset(dataset, num_samples, title):
    """
    Visualize the dataset with images and their corresponding masks.

    Parameters:
    - dataset: The TensorFlow dataset to visualize.
    - num_samples: Number of samples to visualize.
    - title: Title for the plot.
    """
    plt.figure(figsize=(15, num_samples * 3))
    for i, (image, mask) in enumerate(dataset.take(num_samples)):
        plt.subplot(num_samples, 2, 2 * i + 1)
        plt.imshow(np.squeeze(image.numpy()), cmap='gray')
        plt.title(f'{title} Image {i+1}')
        plt.axis('off')

        plt.subplot(num_samples, 2, 2 * i + 2)
        plt.imshow(np.squeeze(mask.numpy()), cmap='gray')
        plt.title(f'{title} Mask {i+1}')
        plt.axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
def display_three_images(mask, image, predicted_mask):
  f, axarr = plt.subplots(1,3)
  axarr[0].imshow(mask)
  axarr[1].imshow(image)
  axarr[2].imshow(predicted_mask)

# Section 5: Execution of Processing Pipeline

## Pipeline Configuration

Set up the processing pipeline by selecting the desired filters and augmentations. This configuration will determine how the images are processed and enhanced.



In [ ]:
pipeline = ProcessingPipeline()
pipeline.add_augmentations([Rotate(), Translate(), Flip(), BrightnessContrast(), RandomGaussianBlur(), MedianBlur()])

base_masks_path = "/content/drive/MyDrive/Images/Masks"
base_inputs_path = "/content/drive/MyDrive/Images/Input"

image_data_manager = ImageDataManager(base_masks_path, base_inputs_path)

## Visualizing

In [ ]:
Visualizer.visualize_objects(image_data_manager.objects)

## Loading resources as `np.array`

In [ ]:
import numpy as np
import cv2

RAW_IMAGE_SIZE = 1200
CROP_SIZE = 120  # RAW_IMAGE_SIZE // 10
IMAGE_SIZE = (CROP_SIZE, CROP_SIZE)
TEST_SET_SIZE_AS_PERCENTAGE = 0.2
BATCH_SIZE = 1

masks_as_np_array = []
images_as_np_array = []

for key in image_data_manager.objects.keys():
    images = image_data_manager.objects[key]['images']
    mask = image_data_manager.objects[key]['mask']

    filtered_images = []

    for image in images:
        if image.shape[1] == RAW_IMAGE_SIZE:
            filtered_images.append(np.transpose(image, (1, 2, 0)))

    image_data_manager.objects[key]['images'] = filtered_images

    image = filtered_images[-1]
    n_crop = image.shape[0] // CROP_SIZE
    images, masks, coordinates = pipeline.run(image, mask, crop_size=CROP_SIZE, n_crop=n_crop, n_augmented=0)

    masks_as_np_array.extend(masks)
    images_as_np_array.extend(images)

print(f"Loaded {len(images_as_np_array)} images")



In [ ]:
images_as_np_array = np.array(images_as_np_array)
masks_as_np_array = np.array(masks_as_np_array)

In [ ]:
masks_as_np_array = np.expand_dims(masks_as_np_array, axis=-1)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(images_as_np_array, masks_as_np_array, test_size=0.2, random_state=42)

## Usefull info

In [ ]:
masks_as_np_array[0].shape

In [ ]:
images_as_np_array[0].shape

In [ ]:
display_three_images(masks_as_np_array[0], images_as_np_array[0], images_as_np_array[0])

In [ ]:
np.asarray(images_as_np_array).shape

## Model

In [ ]:
pretrained_model = tf.keras.applications.ResNet50(
    include_top=False,
    input_shape=(120, 120, 3),
    pooling='max',
    weights='imagenet'
)

In [ ]:
layer_name_to_stop_at = 'conv5_block3_out'
truncated_model = Model(
    inputs=pretrained_model.input,
    outputs=pretrained_model.get_layer(layer_name_to_stop_at).output
)
for layer in truncated_model.layers:
    layer.trainable = False

In [ ]:
resnet_model = Sequential()
resnet_model.add(truncated_model) # 4x4
resnet_model.add(Conv2DTranspose(1024, kernel_size=3, strides=2, padding='same', activation='relu')) # 8x8
resnet_model.add(Conv2DTranspose(512, kernel_size=3, strides=2, padding='same', activation='relu')) # 16x16
resnet_model.add(Conv2DTranspose(256, kernel_size=3, strides=2, padding='same', activation='relu')) # 32x32
resnet_model.add(Conv2DTranspose(128, kernel_size=3, strides=2, padding='same', activation='relu')) # 64x64
resnet_model.add(Conv2DTranspose(64, kernel_size=3, strides=2, padding='same', activation='relu')) # 128x128
# Add final Conv2DTranspose layers to reach 120x120
resnet_model.add(Conv2DTranspose(32, kernel_size=3, strides=1, padding='same', activation='relu'))    # Output shape: 128x128x32
resnet_model.add(Conv2D(1, kernel_size=9, strides=1, padding='valid', activation='sigmoid'))  # Output shape: 120x120x1


In [ ]:
optimizer = K.optimizers.Adam(learning_rate=0.01)
resnet_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# history = resnet_model.fit(train_ds, validation_data=validation_ds, epochs=10)
history = resnet_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

## Model infos


In [ ]:
resnet_model.summary()

In [ ]:
truncated_model.summary()

In [ ]:
pretrained_model.summary()

## Results

In [ ]:
fig, ax1 = plt.subplots(figsize=(12, 6))

# Plotting training & validation loss
ax1.plot(history.history['loss'], color='blue', label='Training Loss')
ax1.plot(history.history['val_loss'], color='blue', linestyle='dashed', label='Validation Loss')
ax1.set_xlabel('Epochs')
ax1.set_ylabel('Loss', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

# Creating a secondary y-axis to plot accuracy
ax2 = ax1.twinx()
ax2.plot(history.history['accuracy'], color='green', label='Training Accuracy')
ax2.plot(history.history['val_accuracy'], color='green', linestyle='dashed', label='Validation Accuracy')
ax2.set_ylabel('Accuracy', color='green')
ax2.tick_params(axis='y', labelcolor='green')

# Adding legends
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines + lines2, labels + labels2, loc='upper right')

plt.title('Training and Validation Loss and Accuracy')
plt.show()

In [ ]:
train_predictions = resnet_model.predict(X_train)
test_predictions = resnet_model.predict(X_test)

In [ ]:
display_three_images(X_test[0], test_predictions[0], y_test[0])

## Saving model

In [ ]:
resnet_model.save('/content/drive/MyDrive/models/resnet_first_try.h5')